In [ ]:
import qiskit
qiskit.__version__

'1.3.2'

In [13]:
# pylint: disable=invalid-name
import matplotlib.pyplot as plt
import numpy as np
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
print("Imports Successful")

Imports Successful


In [14]:
pip install qiskit-algorithms

Note: you may need to restart the kernel to use updated packages.


In [21]:
import qiskit_algorithms

In [20]:
qiskit_algorithms.

NameError: name 'Shors' is not defined

In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from numpy.random import randint
from math import gcd
import numpy as np
from fractions import Fraction

def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(n_count + 4, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>

    for q in range(n_count):
        qc.append(c_amod15(a, 2**q), 
                  [q] + [i+n_count for i in range(4)])

    # Do inverse-QFT to finish the period finding
    qc.append(qft_dagger(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))
    # Simulate and return the result
    aer_sim = Aer.get_backend('aer_simulator')
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc)
    result = aer_sim.run(qobj).result()
    counts = result.get_counts()
    return counts

def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2, 13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7, 8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(0,2)
            U.swap(1,3)
        if a in [7, 11, 8, 13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

N = 15
a = 7
counts = qpe_amod15(a)
print("Counts:", counts)
phases = [int(output, 2)/256 for output in counts]
print("Phases:", phases)
frac = [Fraction(phase).limit_denominator(15) for phase in phases]
print("Fractions:", frac)
r = frac[0].denominator
print("Order r:", r)
if r % 2 != 0:
    print("r is odd, try again.")
else:
    guess1 = gcd(a**(r//2)-1, N)
    guess2 = gcd(a**(r//2)+1, N)
    print(f"Factors of {N}: {guess1} and {guess2}")
